In [24]:
from sklearn.datasets import load_boston
!pip install optuna

from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
import lightgbm as lgb
import xgboost as xgb
# sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler

import sklearn.metrics

from mlxtend.classifier import StackingCVClassifier
!pip install vecstack
from vecstack import stacking

from itertools import combinations 


import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import random
sns.set()

In [3]:
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors

import operator as op 
from itertools import combinations


In [4]:
X, y = load_boston(return_X_y=True)
X.shape

(506, 13)

Creating Features 

In [5]:

X, dy = load_boston(return_X_y=True)
DX =  pd.DataFrame(X) # copy of the data for later
X =  pd.DataFrame(X) # copy to destroy


def brute_force_feat(X):

  comb= list(combinations(X.columns,3))

  def operations(x,a,b):
    if (x == '+'): 
      d =  op.add(a,b) 
    if (x == '-'): 
      d =  op.sub(a,b) 
    if (x == '*'): 
      d =  op.mul(a,b)     
    if (x == '/'): # divide by 0 error
      d =  op.truediv(a,(b + 1e-20)) 
    return d

  print('Operator Features')
  for x in ['*','/','+','-']:
    for y in ['*','/','+','-']:
      for i in comb:
        a = X.iloc[:,i[0]].values
        b = X.iloc[:,i[1]].values
        c = X.iloc[:,i[2]].values
        d = operations(x,a,b)
        e = operations(y,d,c)
        X[f'{i[0]}{x}{i[1]}{y}{i[2]}'] = e
        X[f'{i[0]}{x}{i[1]}'] = d

  X = X.loc[:,~X.columns.duplicated()]

  print('Dimensionality Features')
  # DIMENSIONALITY FEATURES
  # 2 is usually a good size, however higher may result in less reward for time taken
  X_TSNE = TSNE(n_components=2).fit_transform(X)
  X_DBSCAN = DBSCAN(eps=3, min_samples=2).fit(X)
  X_PCA = PCA(n_components=2).fit_transform(X)
  # Depending on how large the dataset is increase or decrease n 
  X_KNN64, indices = NearestNeighbors(n_neighbors=64, algorithm='ball_tree').fit(X).kneighbors(X)
  X = np.c_[X, X_KNN64] # NUMPY
  X = np.c_[X, X_PCA]
  X = np.c_[X, X_DBSCAN.labels_]
  X = np.c_[X, X_TSNE]
  X = np.where(np.isnan(X), 0, X)


  # Selecting best 50 columns 
  from sklearn.ensemble import RandomForestRegressor
  # PANDAS
  # SX = X.sample(n=500) # Sample, etc

  # NUMPY
  print('No. Features:', np.size(X,1))
  np.random.seed(42)
  # idx = np.random.randint(np.size(X,0), size=500) # make this 50% of your data just so it runs faster
  # X = [idx,:]
  # dy = dy.values[idx]
  # print(np.any(np.isnan(X)))
  # print(np.all(np.isfinite(X)))
  X = np.nan_to_num(X.astype(np.float32)) # prevents too large values error

  # SX IS A SAMPLE OF X
  SX = X
  sy = dy # other methods if this gives error #.values.ravel() #np.array(dy)[idx.astype(int)] #dy[idx]
  
  model_rfi = RandomForestRegressor(n_jobs = -1, random_state=42)

  model_rfi.fit(SX, sy)
  print('Done!')

  return X, DX, model_rfi

# CX is the new features data | DX is the original data  | Model_rfi contains the feature importance
CX, DX, model_rfi = brute_force_feat(X)

Operator Features
Dimensionality Features
No. Features: 4922
Done!


In [7]:
def scaler_fuc(scaler, f, CX, DX):

  # order by most important features and take the f most important features
  fs_indices_rfi = np.argsort(model_rfi.feature_importances_)[::-1][0:f]
  CX = CX[:,fs_indices_rfi]
  X = np.c_[DX, CX]
  # delete duplicates
  _, unq_col_indices = np.unique(X,return_index=True,axis=1)
  X = X[:,unq_col_indices]

  train_x, valid_x, train_y, valid_y = train_test_split(X, dy,
                                              test_size=0.25, random_state = 123)
  if (scaler == 'minmax'):
    scaler = MinMaxScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    scaler.fit(valid_x)
    valid_x = scaler.transform(valid_x)

  if (scaler == 'stand'):
    scaler = StandardScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    scaler.fit(valid_x)
    valid_x = scaler.transform(valid_x)

  if (scaler == 'log'):
    transformer = FunctionTransformer(np.log1p, validate=True)
    train_x = transformer.transform(train_x)
    valid_x = transformer.transform(valid_x)
    # prevent log 0 error
    np.seterr(divide = 'ignore')  # invalid value encountered in log1p
    train_x = np.where(np.isneginf(train_x), 0, train_x)
    valid_x = np.where(np.isneginf(valid_x), 0, valid_x)
    train_x = np.where(np.isinf(train_x), 0, train_x)
    valid_x = np.where(np.isinf(valid_x), 0, valid_x)
    train_x = np.where(np.isnan(train_x), 0, train_x)
    valid_x = np.where(np.isnan(valid_x), 0, valid_x)

  # turning these train/tests into lgb/xgb datasets
  dtrain_gbm = lgb.Dataset(train_x, label=train_y)
  dvalid_gbm = lgb.Dataset(valid_x, label=valid_y)

  dtrain_xbg = xgb.DMatrix(train_x, label=train_y)
  dvalid_xbg = xgb.DMatrix(valid_x, label=valid_y)
  return train_x, valid_x, train_y, valid_y,  dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg


In [19]:
models = ['lreg', 'las', 'rid','ada', 'et', 'rf', 'xgb', 'gbm'] # when you want to try all of them/ iteration: 1 
# models = ['ada', 'et', 'rf', 'xgb', 'gbm'] # selecting the 5 best models from iteration 1 (by observing the log) / iteration: 2
# models = ['ada', 'et', 'xgb'] 
comb = combinations(models, 3) 
comb = list(comb)

In [20]:

class Objective:

    def __init__(self):
        self.best_gbm = None
        self._gbm = None
        self.best_xgb = None
        self._xgb = None
        self.predictions = None
        self.fpredictions = None

    def __call__(self, trial):

        i = trial.suggest_int("combos", 0, (len(comb)-1))
 
        gbm_preds = 0
        gbm_rmse = float('inf')
        xgb_preds = 0
        xgb_rmse = float('inf')
        rf_preds = 0
        rf_rmse = float('inf')
        et_preds = 0
        et_rmse = float('inf')
        ada_preds = 0
        ada_rmse = float('inf')
        lreg_preds = 0
        lreg_rmse = float('inf')
        las_preds = 0
        las_rmse = float('inf')
        rid_preds = 0
        rid_rmse = float('inf')

        ###############################################################################
        #                                 . Scaling                                   #
        ###############################################################################
        # If you want 1 scaled data for everything use this and comment out, other wise leave this commented
        # scaler = trial.suggest_categorical("Scaler", ['minmax','stand','log'])
        # train_x, valid_x, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(scaler)
        ###############################################################################
        #                            . Linear Regression                              #
        ###############################################################################
        if any(x == 'lreg' for x in comb[i]):
          lr_f = trial.suggest_int("lr_features", 0, 20)
          lr_scaler = trial.suggest_categorical("lr_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(lr_scaler, lr_f, CX, DX)
          lreg = LinearRegression().fit(train_x, train_y)
          lreg_preds = lreg.predict(valid_x)
          lreg_rmse = sklearn.metrics.mean_squared_error(valid_y, lreg_preds, squared = False)
        ###############################################################################
        #                                 . Lasso                                     #
        ###############################################################################
        if any(x == 'las' for x in comb[i]):
          las_f = trial.suggest_int("las_features", 0, 20)
          las_scaler = trial.suggest_categorical("las_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(las_scaler, las_f, CX, DX)
          alphal = trial.suggest_loguniform("alphal", 1e-8, 1.0)
          las = linear_model.Lasso(alpha=alphal).fit(train_x, train_y)
          las_preds = las.predict(valid_x)
          las_rmse = sklearn.metrics.mean_squared_error(valid_y, las_preds, squared = False)
        ###############################################################################
        #                                 . Ridge                                     #
        ###############################################################################
        if any(x == 'rid' for x in comb[i]):
          rid_f = trial.suggest_int("rid_features", 0, 20)
          rid_scaler = trial.suggest_categorical("rid_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(rid_scaler, rid_f, CX, DX)
          alphar = trial.suggest_loguniform("alphar", 1e-8, 1.0)
          rid = Ridge(alpha=alphar).fit(train_x, train_y)
          rid_preds = rid.predict(valid_x)
          rid_rmse = sklearn.metrics.mean_squared_error(valid_y, rid_preds, squared = False)
        ###############################################################################
        #                               . Ada Boost                                   #
        ###############################################################################
        if any(x == 'ada' for x in comb[i]):
          ada_f = trial.suggest_int("ada_features", 0, 20)
          ada_scaler = trial.suggest_categorical("ada_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(ada_scaler, ada_f, CX, DX)
          ada_ne = trial.suggest_int("ada_ne", 1, 500) #500
          ada_lr = trial.suggest_loguniform("ada_lr", 0.05, 1)
          ada = AdaBoostRegressor(n_estimators = ada_ne, learning_rate = ada_lr,
                                     random_state=0).fit(train_x, train_y)
          ada_preds = ada.predict(valid_x)
          ada_rmse = sklearn.metrics.mean_squared_error(valid_y, ada_preds, squared = False)
        ###############################################################################
        #                              . Extra Trees                                  #
        ###############################################################################
        if any(x == 'et' for x in comb[i]):
          et_f = trial.suggest_int("et_features", 0, 20)
          et_scaler = trial.suggest_categorical("et_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(et_scaler,et_f, CX, DX)
          et_md = trial.suggest_int("et_max_depth", 1, 100)
          et_ne = trial.suggest_int("et_ne", 1, 500) #1000
          et = ExtraTreesRegressor(max_depth=et_md, n_estimators = et_ne,
                                     random_state=0).fit(train_x, train_y)
          et_preds = et.predict(valid_x)
          et_rmse = sklearn.metrics.mean_squared_error(valid_y, et_preds, squared = False)
        ###############################################################################
        #                             . Random Forest                                 #
        ###############################################################################
        if any(x == 'rf' for x in comb[i]):
          rf_f = trial.suggest_int("rf_features", 0, 20)
          rf_scaler = trial.suggest_categorical("rf_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(rf_scaler,rf_f, CX, DX)
          rf_md = trial.suggest_int("rf_max_depth", 1, 100)
          rf_ne = trial.suggest_int("rf_ne", 1, 500) #1000
          rf = RandomForestRegressor(max_depth=rf_md, n_estimators = rf_ne,n_jobs = -1,
                                     random_state=0).fit(train_x, train_y)
          rf_preds = rf.predict(valid_x)
          rf_rmse = sklearn.metrics.mean_squared_error(valid_y, rf_preds, squared = False)
        ###############################################################################
        #                                 . XGBoost                                   #
        ###############################################################################
        if any(x == 'xgb' for x in comb[i]): 
          xgb_f = trial.suggest_int("xgb_features", 0, 20)
          xgb_scaler = trial.suggest_categorical("xgb_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(xgb_scaler,xgb_f, CX, DX)
          xgb_param = {
              "silent": 1,
              "objective": "reg:squarederror",
              "eval_metric": "rmse",
              "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
              "lambda": trial.suggest_loguniform("lambda", 1e-8, 1.0),
              "alpha": trial.suggest_loguniform("alpha", 1e-8, 1.0),
          }
          if xgb_param["booster"] == "gbtree" or xgb_param["booster"] == "dart":
              xgb_param["max_depth"] = trial.suggest_int("max_depth", 1, 100)
              xgb_param["eta"] = trial.suggest_loguniform("eta", 1e-8, 1.0)
              xgb_param["gamma"] = trial.suggest_loguniform("gamma", 1e-8, 1.0)
              xgb_param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
          if xgb_param["booster"] == "dart":
              xgb_param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
              xgb_param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
              xgb_param["rate_drop"] = trial.suggest_loguniform("rate_drop", 1e-8, 1.0)
              xgb_param["skip_drop"] = trial.suggest_loguniform("skip_drop", 1e-8, 1.0)
          # Add a callback for pruning.
          xgb_pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse" )
          xgb_ = xgb.train(xgb_param, dtrain_xbg, evals=[(dvalid_xbg, "validation")], verbose_eval=False, callbacks=[xgb_pruning_callback])
          xgb_preds = xgb_.predict(dvalid_xbg)
          xgb_rmse = sklearn.metrics.mean_squared_error(valid_y, xgb_preds, squared = False)
          self._xgb = xgb_
        ###############################################################################
        #                          . Light Gradient Boosting                          #
        ###############################################################################
        if any(x == 'gbm' for x in comb[i]):
          gbm_f = trial.suggest_int("gbm_features", 0, 20)
          gbm_scaler = trial.suggest_categorical("gbm_Scaler", ['minmax','stand','log'])
          train_x, valid_x, train_y, valid_y, dtrain_gbm, dvalid_gbm, dtrain_xbg, dvalid_xbg = scaler_fuc(gbm_scaler,gbm_f, CX, DX)
          gbm_param = {
            'objective': 'regression',
            'metric': 'rmse',
              "verbosity": -1,
              "boosting_type": "gbdt",
              "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10), 
              "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10),
              "num_leaves": trial.suggest_int("num_leaves", 2, 256), 
              "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0), 
              "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
              "bagging_freq": trial.suggest_int("bagging_freq", 1, 7), 
              "min_child_samples": trial.suggest_int("min_child_samples", 2, 20), 
          }
          # Add a callback for pruning.
          gbm_pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
          gbm = lgb.train(gbm_param, dtrain_gbm, valid_sets=[dvalid_gbm], verbose_eval=False, callbacks=[gbm_pruning_callback])
          gbm_preds = gbm.predict(valid_x)
          gbm_rmse = sklearn.metrics.mean_squared_error(valid_y, gbm_preds, squared = False)
          self._gbm = gbm
        ###############################################################################
        #                            . Stacking Strategy                              #
        ###############################################################################

        # strat = trial.suggest_int("strat", 0, 1)

        # if (strat == 0): # taking mean
        preds = (gbm_preds + xgb_preds + rf_preds + \
        las_preds + ada_preds + et_preds + \
        lreg_preds + rid_preds) / 3

        # if (strat == 1): # Making an equation
        # a = trial.suggest_uniform("a", 0.1, 0.9)
        # b = trial.suggest_uniform("b", 0.1, 0.9)
        # c = trial.suggest_uniform("c", 0.1, 0.9)
        # d = trial.suggest_uniform("d", 0.1, 0.9)

        # def custom_sort(t):
        #   return t[0]

        # R = [[gbm_rmse,gbm_preds] , [xgb_rmse,xgb_preds], [rf_rmse, rf_preds], \
        #     [las_rmse,las_preds], [ada_rmse,ada_preds],[et_rmse,et_preds],\
        #     [lreg_rmse,lreg_preds], [rid_rmse,rid_preds]]
        # R.sort(key=custom_sort) # low to high
        # m1 = R[0][1]
        # m2 = R[1][1]
        # m3 = R[2][1]
        # # Using list comprehension. This may result in a divide by 0 error when rounding occurs.
        # # A fix would be to use a more precise library
        # preds = [ (a*((x**b) + (y**c))) + (d*z) for x,y,z in zip(m1,m2,m3) ]
        self.predictions = preds
        rmse = sklearn.metrics.mean_squared_error(valid_y, preds, squared = False)
        return rmse

    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_gbm = self._gbm
            self.best_xgb = self._xgb
            self.fpredictions = self.predictions

In [21]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) # for log error

import optuna
objective = Objective()

# Setting SEED 
from optuna.samplers import TPESampler
sampler = TPESampler(seed=10)

study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize",
    sampler=sampler
)
study.optimize(objective, n_trials=1000, callbacks=[objective.callback]) # change this to 500 + 

print("Best trial:")
trial = study.best_trial

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

best_gbm = objective.best_gbm
best_xgb = objective.best_xgb

[I 2020-07-13 04:32:50,226] Finished trial#0 with value: 3.822296998606017 with parameters: {'combos': 9, 'et_features': 4, 'et_Scaler': 'minmax', 'et_max_depth': 29, 'et_ne': 474, 'xgb_features': 16, 'xgb_Scaler': 'stand', 'booster': 'gblinear', 'lambda': 0.004716980476847206, 'alpha': 1.3320760321578133e-06, 'gbm_features': 10, 'gbm_Scaler': 'minmax', 'lambda_l1': 2.2563205837462292e-07, 'lambda_l2': 0.014268090793125176, 'num_leaves': 102, 'feature_fraction': 0.4023689597967487, 'bagging_fraction': 0.707315358031466, 'bagging_freq': 4, 'min_child_samples': 3}. Best is trial#0 with value: 3.822296998606017.
[I 2020-07-13 04:32:50,733] Finished trial#1 with value: 4.013429985935784 with parameters: {'combos': 8, 'rid_features': 4, 'rid_Scaler': 'log', 'alphar': 0.006599090886310522, 'xgb_features': 13, 'xgb_Scaler': 'stand', 'booster': 'gblinear', 'lambda': 2.0386327434753964e-05, 'alpha': 0.016975808449119403, 'gbm_features': 12, 'gbm_Scaler': 'stand', 'lambda_l1': 0.0036299680810414

Best trial:
  Params: 
    combos: 3
    ada_features: 7
    ada_Scaler: stand
    ada_ne: 287
    ada_lr: 0.9991821582026199
    et_features: 0
    et_Scaler: stand
    et_max_depth: 10
    et_ne: 102
    xgb_features: 10
    xgb_Scaler: log
    booster: gbtree
    lambda: 0.4201479647656314
    alpha: 8.337801160413445e-06
    max_depth: 42
    eta: 0.40902586593808343
    gamma: 0.00017385274444494498
    grow_policy: depthwise


In [22]:
comb[3]

('ada', 'et', 'xgb')

best result so far:


**BOSTON HOUSE** 

2.8669 : ('ada', 'et', 'xgb')

```
  Params: 
    combos: 3
    ada_features: 7
    ada_Scaler: stand
    ada_ne: 287
    ada_lr: 0.9991821582026199
    et_features: 0
    et_Scaler: stand
    et_max_depth: 10
    et_ne: 102
    xgb_features: 10
    xgb_Scaler: log
    booster: gbtree
    lambda: 0.4201479647656314
    alpha: 8.337801160413445e-06
    max_depth: 42
    eta: 0.40902586593808343
    gamma: 0.00017385274444494498
    grow_policy: depthwise
```






In [23]:
objective.fpredictions 